In [23]:
import os
import sys
import logging

from dotenv import load_dotenv
from llama_index.core import PromptTemplate
from llama_index.core.prompts import PromptType
from llama_index.core.tools import ToolMetadata
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.selectors import LLMSingleSelector

In [24]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(formatter)
logger.addHandler(handler)

load_dotenv()

True

In [25]:
MODEL_ID=os.environ.get("MODEL_ID", "anthropic.claude-3-haiku-20240307-v1:0")
AWS_PROFILE_NAME=os.environ.get("AWS_PROFILE_NAME", None)

In [26]:
# choices as a list of strings
choices = [
    "Useful for questions related to mobile game",
    "Useful for questions related to PC game",
]

In [27]:
# setup llm
llm = BedrockConverse(
    model=MODEL_ID,
    max_tokens=1024 * 2,
    temperature=0.4,
    profile_name=AWS_PROFILE_NAME,
)

In [41]:
prompt_template_str = (
    "Some choices are given below. It is provided in a numbered list (1 to {num_choices}), where each item in the list corresponds to a summary.\n"
    "---------------------\n"
    "{context_list}\n"
    "---------------------\n"
    "Using only the choices above and not prior knowledge, return the choice that is most relevant to the question: '{query_str}'\n"
    "If the question can not be classified by context_list, please use number 9."
)

In [42]:
selector = LLMSingleSelector.from_defaults(
    llm=llm,
    prompt_template_str=prompt_template_str,
)

In [43]:
selector.get_prompts()

{'prompt': PromptTemplate(metadata={'prompt_type': <PromptType.SINGLE_SELECT: 'single_select'>}, template_vars=['num_choices', 'context_list', 'query_str'], kwargs={}, output_parser=<llama_index.core.output_parsers.selection.SelectionOutputParser object at 0x173181e40>, template_var_mappings=None, function_mappings=None, template="Some choices are given below. It is provided in a numbered list (1 to {num_choices}), where each item in the list corresponds to a summary.\n---------------------\n{context_list}\n---------------------\nUsing only the choices above and not prior knowledge, return the choice that is most relevant to the question: '{query_str}'\nIf the question can not be classified by context_list, please use number 9.")}

### Choice 0 - Game

In [44]:
%%time

selector_result = selector.select(
    choices, query="리니지W 시작한지 이틀 되었는데, 초보자 공략방법 좀 알려줘."
)
print(selector_result.selections)

[SingleSelection(index=0, reason='The question is about a mobile game (리니지W), so the most relevant choice is (1) Useful for questions related to mobile game.')]
CPU times: user 8.76 ms, sys: 3.6 ms, total: 12.4 ms
Wall time: 890 ms


### Choice 1 - PC game

In [45]:
%%time

selector_result = selector.select(
    choices, query="리니지2 시작한지 이틀 되었는데, 초보자 공략방법 좀 알려줘."
)
print(selector_result.selections)

[SingleSelection(index=1, reason='The question is about strategies for a PC game, Lineage 2, which is not a mobile game.')]
CPU times: user 6.99 ms, sys: 2.26 ms, total: 9.26 ms
Wall time: 921 ms


### Invalid Choice

In [46]:
%%time

selector_result = selector.select(
    choices, query="탁구 시작한지 이틀 되었는데, 초보자 공략방법 좀 알려줘."
)
print(selector_result.selections)

[SingleSelection(index=8, reason='The given question is about tips for a beginner in table tennis, which is not related to either mobile games or PC games.')]
CPU times: user 8.37 ms, sys: 3.55 ms, total: 11.9 ms
Wall time: 888 ms
